This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

# Tested that this correctly printed my api key, but not committing this here

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import math

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key={API_KEY}'
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()
print(type(data))
data

<class 'dict'>


{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-01-01',
  'end_date': '2017-01-02',
  'frequency': 'daily',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# store params as variables to construct the url request
start='2017-01-01'
end='2017-12-31'

url = f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date={start}&end_date={end}&api_key={API_KEY}'

r = requests.get(url)

In [6]:
data = r.json()

# verify data is a python dict
print(type(data))

#it appears that there is a top level, single key of 'dataset_data', so get the keys of the nested dict
print(data['dataset_data'].keys())

# store this in another variable
inner_dict = data['dataset_data']


<class 'dict'>
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [7]:
# The 'data' key contains the pricing data we are interested in. 
# 'column_names' corresponds to the data in the 'data' dict of lists
column_names = inner_dict['column_names']
day_1 = inner_dict['data'][0]
# Just an example test to create a dict of column names and their corresponding values
daily_data_dict = dict(zip(column_names, day_1))
print(daily_data_dict)

{'Date': '2017-12-29', 'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0, 'Turnover': 1792304.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}


In [8]:
# The data is returned in an array of arrays, one for each day
# To make it easier to work with, without Pandas, convert to a list of dictionaries
daily_data = [dict(zip(column_names, day)) for day in inner_dict['data']]

In [10]:
# Calculate what the highest and lowest opening prices were for the stock in this period.
def highest_lowest_opening_price(stock_data):
    # initialize highest and lowest prices on first day
    highest = stock_data[0]['Open']
    lowest = stock_data[0]['Open']
    for dict in stock_data:
        if type(dict['Open']) == float and dict['Open'] > highest:
            highest = dict['Open']
        if type(dict['Open']) == float and dict['Open'] < lowest:
            lowest = dict['Open']
    return (highest, lowest)



In [11]:
# My method determined that the highest open price was 53.11, and the lowest was 34.0
highest_lowest_opening_price(daily_data)

(53.11, 34.0)

In [20]:
# Going to try to do this the pythonic way with a list comprehension 
highest = max((trade['Open'] for trade in daily_data if type(trade['Open']) == float))
lowest = min((trade['Open'] for trade in daily_data if type(trade['Open']) == float))
print(highest, lowest)

53.11 34.0


In [12]:
# What was the largest change in any one day (based on High and Low price)?
def get_max_single_day_change(stock_data):
    biggest_change = stock_data[0]['High']-stock_data[0]['Low']
    for dict in stock_data:
        if type(dict['High']) == float and type(dict['Low']) == float and dict['High']-dict['Low'] > biggest_change:
            biggest_change = dict['High'] - dict['Low']
    return biggest_change


In [13]:
# My method shows a biggest change of 2.81 in a single day
get_max_single_day_change(daily_data)

2.8100000000000023

In [22]:
max_change = max((trade['High']-trade['Low'] for trade in daily_data if type(trade['High']) == float and type(trade['Low']) == float))
print(max_change)

2.8100000000000023


In [14]:
# What was the largest change between any two days (based on Closing Price)?
def largest_two_day_change(stock_data):
    # initialize with 0 just in case the second dict is a None Type (or something)
    largest_change = 0
    for i in range(len(stock_data)-1):
        if type(stock_data[i]['Close']) == float and type(stock_data[i+1]['Close']) == float: 
            temp = abs(stock_data[i]['Close'] - stock_data[i+1]['Close'])
            if temp > largest_change: 
                largest_change = temp
    return largest_change

In [15]:
# My method returns 2.56 for the largest change in two days, base on closing price
largest_two_day_change(daily_data)

2.559999999999995

In [16]:
# What was the average daily trading volume during this year?
# I got 89124.33725490196 for the average trading volume for the year
ave_trading_vol = (sum(trade['Traded Volume'] for trade in daily_data if type(trade['Traded Volume']) == float))/len(daily_data)
ave_trading_vol

89124.33725490196

In [17]:
# (Optional) What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)
def get_median(stock_data):
    # This will not be efficient, but thankfully it's not too much data
    traded_vol = [day['Traded Volume'] for day in stock_data]
    traded_vol.sort()
    # Get the median value, or the average of the two central values if an even length
    arr_len = len(traded_vol)    
    if arr_len%2 != 0:
        return traded_vol[math.floor(arr_len/2)]
    else:
        return (traded_vol[arr_len/2 - 1] + traded_vol[arr_len/2 + 1])/2


In [18]:
# I got a median value of 76286.0 shares
get_median(daily_data)

76286.0